In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import matplotlib.pyplot as plt

In [2]:
# Chemin vers le fichier GZIP
gzip_file_path = "C:/Users/HanSAriEle/OneDrives/Desktop/PROJET_TUTORE/database_quality_ofair.csv.gz"

# Lire le fichier CSV directement depuis le fichier GZIP
data = pd.read_csv(gzip_file_path, compression='gzip')

# Afficher les premières lignes du DataFrame pour vérifier
data.head()

,Date,City,Country_Name,Continent,aqi,co,neph,no2,o3,pm10,pm25,so2,Categorie
0,2019-01-01,Addis Ababa,Ethiopia,Africa,38.0,1.933333,23.333333,7.333333,16.533333,15.333333,38.000000,1.266667,Bon
1,2019-01-01,Adelaide,Australia,Oceania,7.0,0.100000,3.666667,0.100000,0.000000,7.000000,6.000000,0.100000,Bon
2,2019-01-01,Barcelona,Spain,Europe,27.0,0.100000,9.666667,17.000000,4.900000,27.000000,27.666667,4.600000,Bon
3,2019-01-01,Bilbao,Spain,Europe,68.0,0.100000,39.000000,12.800000,3.300000,23.000000,68.000000,4.000000,Modéré
4,2019-01-01,Bogotá,Colombia,South America,34.0,4.600000,8.666667,2.300000,8.300000,17.000000,34.000000,0.300000,Bon


In [27]:
# Séparer les features (variables prédictives) et la classe (variable cible)
X = data.drop(['Country_Name', 'Date', 'Categorie', 'City', 'Continent'], axis=1)  # Supprimer les colonnes non pertinentes
y = data['City']  # Variable cible (classe)

# Sélectionner uniquement les colonnes numériques pour l'agrégation
numeric_columns = ['aqi', 'co', 'neph', 'no2', 'o3', 'pm10', 'pm25', 'so2']
non_numeric_columns = ['City', 'Country_Name', 'Continent']

# Vérifier que les colonnes sélectionnées existent dans les données
for col in numeric_columns:
    if col not in data.columns:
        print(f"Warning: {col} not found in data columns")


# Agréger les données par ville (en utilisant les colonnes numériques)
agg_data_numeric = data.groupby('City')[numeric_columns].mean().reset_index()

# Conserver les informations non numériques pour les premières occurrences
agg_data_non_numeric = data[non_numeric_columns].drop_duplicates(subset=['City']).reset_index(drop=True)

# Combiner les données agrégées avec les informations non numériques
agg_data = pd.merge(agg_data_non_numeric, agg_data_numeric, on='City')


# Standardiser les données
X = agg_data[numeric_columns]
scaler = StandardScaler()
df_normalized = scaler.fit_transform(X)

In [28]:
# Analyse Factorielle Discriminante (AFD)
lda = LinearDiscriminantAnalysis(n_components=2)  # Vous pouvez ajuster le nombre de composantes selon vos besoins
X_lda = lda.fit_transform(X_scaled, y)

In [30]:
# Représentation graphique des individus sur le premier axe discriminant
plt.figure(figsize=(12, 8))
targets = agg_data['Cluster'].unique()
colors = ['b', 'r']  # Couleurs pour chaque classe (à ajuster selon vos clusters)
for target, color in zip(targets, colors):
    indices_to_keep = y == target
    plt.scatter(X_lda[indices_to_keep], np.zeros_like(X_lda[indices_to_keep]), c=color, label=f'Cluster {target}', alpha=0.7)

# Ajouter des étiquettes pour chaque point (ville)
for i, city in enumerate(agg_data['Cluster']):
    plt.annotate(city, (X_lda[i], 0), alpha=0.7)

plt.xlabel('Premier Axe Discriminant')
plt.title('Représentation des individus sur le premier axe discriminant avec AFD')
plt.legend()
plt.grid()
plt.show()

KeyError: 'Cluster'

<Figure size 1200x800 with 0 Axes>